In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [23]:
df = pd.read_csv('/content/Restaurant_Reviews.csv')
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [24]:
df = df.drop(['Liked'], axis=1)

In [19]:
import torch


In [8]:
torch.cuda.empty_cache()

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

model = AutoModelForCausalLM.from_pretrained(
  "SeaLLMs/SeaLLMs-v3-1.5B-Chat",
  torch_dtype=torch.bfloat16,
  device_map=device
)
tokenizer = AutoTokenizer.from_pretrained("SeaLLMs/SeaLLMs-v3-1.5B-Chat")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/767 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/211 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [47]:
def chat_completion(sentence):
  messages = [
    {"role": "system", "content": "You are only allowed to answer in one of these three ways [neutral, negative, positive]. You must not include any explanations or other words in your answer and must not include a full stop in your answer. Also, your answer must not start with a capital letter."},
    {"role": "user", "content": f'Answer the one of sentiment of the given Review. Review: {sentence}'}
  ]
  text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  model_inputs = tokenizer([text], return_tensors="pt").to(device)
  generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512, do_sample=True, eos_token_id=tokenizer.eos_token_id)
  generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]
  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
  result =response[0]
  return result

In [59]:
df['sentiment'] = df['Review'].apply(chat_completion)

In [60]:
df

,Review,sentiment
0,Wow... Loved this place.,Positive
1,Crust is not good.,negative
2,Not tasty and the texture was just nasty.,negative
3,Stopped by during the late May bank holiday of...,positive
4,The selection on the menu was great and so wer...,positive
...,...,...
995,I think food should have flavor and texture an...,negative
996,Appetite instantly gone.,negative
997,Overall I was not impressed and would not go b...,negative
998,"The whole experience was underwhelming, and I ...",negative


In [61]:
df['sentiment'] = df['sentiment'].str.replace('.', '', regex=False).str.lower()

In [66]:
df['sentiment'] = df['sentiment'].str.replace('\n', '', regex=False)
df['sentiment'] = df['sentiment'].str.replace('[', '', regex=False).str.replace(']', '', regex=False)

In [71]:
df = df[df['sentiment'].isin(['neutral', 'negative', 'positive'])]

In [72]:
df['sentiment'].value_counts()

,count
sentiment,
negative,486
positive,444
neutral,64


In [73]:
df.to_csv('sentiment.csv', index=False)